# Forecast Alumn

- Grupo 4
- Nivel de granularidad: Periodo, Sede, CodigoCurso, Horario
- Algoritmo: Regresión lineal

In [2]:
# %pip install azure-ai-ml
# %pip install openpyxl
# %pip install azureml-fsspec azure-identity
# %pip install -U azureml-fsspec

In [20]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.ai.ml import command
from azure.ai.ml import Input, Output
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import pandas as pd
from pathlib import Path
from unidecode import unidecode
from datetime import datetime
from dateutil.relativedelta import relativedelta

## Connect 

In [21]:
# Initialize MLClient
ml_client = MLClient(
    credential=DefaultAzureCredential()
)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [23]:
# Upload local folder to datastore and create data asset
local_folder_path = "../data/rawdata"  # Path to your local folder
data_asset_name = "rawdata_asset"
datastore_name = "rawdata"

In [25]:
data_asset = Data(
    path=local_folder_path,                      # local file or folder
    type=AssetTypes.URI_FOLDER,          # or URI_FILE
    description="Training dataset",
    name=data_asset_name,
    version="1",
    datastore=datastore_name        # 👈 TARGET DATASTORE
)




In [27]:
ml_client.data.create_or_update(data_asset)

In [ ]:
# Path in blob storage after upload (e.g., "folder/subfolder")
remote_folder_path = "my_remote_folder"

# Use azureml:// URI format (automatically resolves credentials)
blob_uri = f"azureml://datastores/workspaceblobstore/paths/{remote_folder_path}/"
# OR direct blob URI (requires SAS token if not public):
# blob_uri = f"https://{storage_account_name}.blob.core.windows.net/{container_name}/{remote_folder_path}?<sas-token>"

my_data = Data(
    name="my_uri_folder_asset_direct",
    type=AssetTypes.URI_FOLDER,
    path=blob_uri,
    description="Data asset without datastore",
)

ml_client.data.create_or_update(my_data)

In [ ]:
try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception:
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential)

In [ ]:
# Subcription ID
sub_id='0fa282e4-ddd7-470a-8f1a-63f922a635a7'
# Resource Group Name
rg_name='forecast-icpna-rg'
# Azure Machine Learning Workspace Name
aml_ws_name='azmlccmlops'
# Base path
base_path = f'azureml://subscriptions/{sub_id}/resourcegroups/{rg_name}/workspaces/{aml_ws_name}/datastores'
# 
path = base_path + "/"+ "/paths/Horarios Merge.xlsx"

df = pd.read_excel(path, storage_options={"credential": credential})

In [ ]:
# configurar el job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    compute="enriquemejiagamarra1",
    display_name="diabetes-pythonv2-train",
    experiment_name="diabetes-training"
)

# enviar el job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In [ ]:
%%writefile $script_folder/move-data.py
# import libraries
import argparse
import pandas as pd
import numpy as np
from pathlib import Path

def main(args):
    # read data
    df = get_data(args.input_data)

    output_df = df.to_csv((Path(args.output_datastore) / "diabetes.csv"), index = False)

# function that reads the data
def get_data(path):
    df = pd.read_csv(path)

    # Count the rows and print the result
    row_count = (len(df))
    print('Analyzing {} rows of data'.format(row_count))
    
    return df

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--input_data", dest='input_data',
                        type=str)
    parser.add_argument("--output_datastore", dest='output_datastore',
                        type=str)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

In [ ]:
# configure input and output
my_job_inputs = {
    "local_data": Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-local:1")
}

my_job_outputs = {
    "datastore_data": Output(type=AssetTypes.URI_FOLDER, path="azureml://datastores/blob_training_data/paths/datastore-path")
}

# configure job
job = command(
    code="./src",
    command="python move-data.py --input_data ${{inputs.local_data}} --output_datastore ${{outputs.datastore_data}}",
    inputs=my_job_inputs,
    outputs=my_job_outputs,
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    compute="yout-compute",
    display_name="move-diabetes-data",
    experiment_name="move-diabetes-data"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)
 